In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'xjnaa652'

In [3]:
wandb.init(id='bovictau', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [8]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m_9.13_e82.torch"), strict=False)

<All keys matched successfully>

In [9]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 55.1 s, sys: 2.01 s, total: 57.1 s
Wall time: 57.2 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
model_stem = "9.17"
START_E = 0

In [11]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 1e-4 #if e < 10 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 83

{'trn_control_loss': 0.01644373, 'trn_headings_loss': 0.00069958, 'trn_curvatures_loss': 0.00215288, 'consistency losses/trn_steer_cost': 0.00464937, 'consistency losses/trn_te_loss': 22.11004829, 'aux losses/trn_pitch_loss': 0.19336319, 'aux losses/trn_yaw_loss': 0.17774677, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7421875, 'consistency losses/trn_torque_loss': 2576.75337934, 'consistency losses/trn_torque_delta_loss': 13.98517146, 'logistical/obs_consumed_per_second': 242.51181102, 'logistical/obs_generated_per_second': 166.7007874, 'logistical/data_consumption_ratio': 1.45415906, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.28212929, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.2265625, 'logistical/obs_generated_per_second': 167.375, 'logistical/data_consumption_ratio': 1.43503753, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01639673, 'trn_headings_loss': 0.00081671, 'trn_curvatures_loss': 0.002715

{'logistical/obs_consumed_per_second': 237.21875, 'logistical/obs_generated_per_second': 166.078125, 'logistical/data_consumption_ratio': 1.42890276, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01395324, 'trn_headings_loss': 0.0006501, 'trn_curvatures_loss': 0.00206587, 'consistency losses/trn_steer_cost': 0.00523399, 'consistency losses/trn_te_loss': 20.706385, 'aux losses/trn_pitch_loss': 0.18300533, 'aux losses/trn_yaw_loss': 0.18883228, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8046875, 'consistency losses/trn_torque_loss': 2938.58652496, 'consistency losses/trn_torque_delta_loss': 13.31917683, 'logistical/max_param': 16.28905487, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 233.390625, 'logistical/obs_generated_per_second': 159.359375, 'logistical/data_consumption_ratio': 1.46727778, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00984403, 'trn_headings_loss': 0.00047795, 'trn_curvatures_loss': 0.00128911, 'consistenc

{'logistical/obs_consumed_per_second': 235.9765625, 'logistical/obs_generated_per_second': 169.2734375, 'logistical/data_consumption_ratio': 1.39300889, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00939831, 'trn_headings_loss': 0.00044378, 'trn_curvatures_loss': 0.00200757, 'consistency losses/trn_steer_cost': 0.00458266, 'consistency losses/trn_te_loss': 21.50891394, 'aux losses/trn_pitch_loss': 0.13311958, 'aux losses/trn_yaw_loss': 0.13679504, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.6953125, 'consistency losses/trn_torque_loss': 2569.28598928, 'consistency losses/trn_torque_delta_loss': 11.99932381, 'logistical/max_param': 16.28923035, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.9453125, 'logistical/obs_generated_per_second': 168.28125, 'logistical/data_consumption_ratio': 1.39695714, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01122981, 'trn_headings_loss': 0.00057171, 'trn_curvatures_loss': 0.00188815, 'cons

{'logistical/obs_consumed_per_second': 237.71875, 'logistical/obs_generated_per_second': 168.6796875, 'logistical/data_consumption_ratio': 1.40941058, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01288125, 'trn_headings_loss': 0.00060739, 'trn_curvatures_loss': 0.00220578, 'consistency losses/trn_steer_cost': 0.00608507, 'consistency losses/trn_te_loss': 20.56500959, 'aux losses/trn_pitch_loss': 0.14452648, 'aux losses/trn_yaw_loss': 0.17406893, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7734375, 'consistency losses/trn_torque_loss': 2626.94907093, 'consistency losses/trn_torque_delta_loss': 12.1711395, 'logistical/max_param': 16.2867794, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 232.4140625, 'logistical/obs_generated_per_second': 168.421875, 'logistical/data_consumption_ratio': 1.38069476, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0099825, 'trn_headings_loss': 0.00048115, 'trn_curvatures_loss': 0.00192692, 'consiste

{'logistical/obs_consumed_per_second': 241.875, 'logistical/obs_generated_per_second': 168.5859375, 'logistical/data_consumption_ratio': 1.43460389, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01197696, 'trn_headings_loss': 0.00051468, 'trn_curvatures_loss': 0.00181147, 'consistency losses/trn_steer_cost': 0.00471427, 'consistency losses/trn_te_loss': 22.30700392, 'aux losses/trn_pitch_loss': 0.15358043, 'aux losses/trn_yaw_loss': 0.1292007, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8359375, 'consistency losses/trn_torque_loss': 2830.62313747, 'consistency losses/trn_torque_delta_loss': 12.69936861, 'logistical/max_param': 16.28388977, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 233.9609375, 'logistical/obs_generated_per_second': 169.8203125, 'logistical/data_consumption_ratio': 1.37705263, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00978079, 'trn_headings_loss': 0.00053908, 'trn_curvatures_loss': 0.0017946, 'consiste

...

{'logistical/obs_consumed_per_second': 240.3125, 'logistical/obs_generated_per_second': 161.3671875, 'logistical/data_consumption_ratio': 1.4901496, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00982422, 'trn_headings_loss': 0.00051955, 'trn_curvatures_loss': 0.00163547, 'consistency losses/trn_steer_cost': 0.00488085, 'consistency losses/trn_te_loss': 21.2090783, 'aux losses/trn_pitch_loss': 0.1434834, 'aux losses/trn_yaw_loss': 0.12021017, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.765625, 'consistency losses/trn_torque_loss': 3164.22909355, 'consistency losses/trn_torque_delta_loss': 11.47617392, 'logistical/max_param': 16.28640366, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.5859375, 'logistical/obs_generated_per_second': 164.3359375, 'logistical/data_consumption_ratio': 1.46404302, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00992985, 'trn_headings_loss': 0.00062593, 'trn_curvatures_loss': 0.00289166, 'consisten

{'logistical/obs_consumed_per_second': 232.9140625, 'logistical/obs_generated_per_second': 163.90625, 'logistical/data_consumption_ratio': 1.42066393, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01105937, 'trn_headings_loss': 0.00055409, 'trn_curvatures_loss': 0.0017855, 'consistency losses/trn_steer_cost': 0.00426795, 'consistency losses/trn_te_loss': 21.14227545, 'aux losses/trn_pitch_loss': 0.10553312, 'aux losses/trn_yaw_loss': 0.12379813, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7890625, 'consistency losses/trn_torque_loss': 2883.41953802, 'consistency losses/trn_torque_delta_loss': 13.23617148, 'logistical/max_param': 16.28717804, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 236.0, 'logistical/obs_generated_per_second': 164.703125, 'logistical/data_consumption_ratio': 1.4324027, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00710204, 'trn_headings_loss': 0.00040426, 'trn_curvatures_loss': 0.00140964, 'consistency l

{'logistical/obs_consumed_per_second': 233.453125, 'logistical/obs_generated_per_second': 164.140625, 'logistical/data_consumption_ratio': 1.42190385, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00972697, 'trn_headings_loss': 0.00051315, 'trn_curvatures_loss': 0.00178565, 'consistency losses/trn_steer_cost': 0.00513889, 'consistency losses/trn_te_loss': 21.3969196, 'aux losses/trn_pitch_loss': 0.1165235, 'aux losses/trn_yaw_loss': 0.11792159, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.6875, 'consistency losses/trn_torque_loss': 2995.11295319, 'consistency losses/trn_torque_delta_loss': 10.0065428, 'logistical/max_param': 16.28383255, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 238.5390625, 'logistical/obs_generated_per_second': 167.5, 'logistical/data_consumption_ratio': 1.42509809, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0102571, 'trn_headings_loss': 0.00047459, 'trn_curvatures_loss': 0.00185652, 'consistency losse

{'logistical/obs_consumed_per_second': 167.0, 'logistical/obs_generated_per_second': 164.0, 'logistical/data_consumption_ratio': 1.01560234, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 157

{'trn_control_loss': 0.0096539, 'trn_headings_loss': 0.00046299, 'trn_curvatures_loss': 0.00143718, 'consistency losses/trn_steer_cost': 0.00421472, 'consistency losses/trn_te_loss': 21.35988502, 'aux losses/trn_pitch_loss': 0.12379694, 'aux losses/trn_yaw_loss': 0.11040807, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8125, 'consistency losses/trn_torque_loss': 2879.35995388, 'consistency losses/trn_torque_delta_loss': 12.6668503, 'logistical/obs_consumed_per_second': 239.64566929, 'logistical/obs_generated_per_second': 159.75590551, 'logistical/data_consumption_ratio': 1.49976202, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.27941132, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.7734375, 'logistical/obs_generated_per_second': 163.1

KeyboardInterrupt: 

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e157.torch")

In [13]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
